In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list names of anyone you worked with on this homework.

# [ER 131] Homework 8: Resampling

The past few assignments (HW5, HW6, and HW7) focused on determining model coefficients using different methods (e.g., ordinary least squares regression with scikit-learn, gradient descent...). This assignment takes a step back, and explores resampling techniques. The point of resampling is to figure out how well a model will perform on data it hasn't seen yet. Generally when we resample, we use different subsamples of our observation data to fit a bunch of models of the same form (e.g., linear models), and then look at how the results differ from model to model. This process allows us to tune hyperparameters and calculate values like the variance of our model coefficients or expected value of the mean squared error.

We've done a version of resampling in the past (Lab 5), where we randomly subset our data into training and testing sets, but the resampling methods that we're covering now are more systematic than random subsets and are designed to ensure that all of the observations are included when training the models.

In this homework, you will work with the cross-validation method was introduced in asynchronous module 8 and in Lab 8. 

A good reference here is ISLR Ch. 5.

### Table of Contents

1. [Project](#proj)<br>
1. [Overfitting](#of)<br>
1. [Cross Validation](#cv)<br> 


----

## Section 1: Project<a name='proj'></a>

***For parts 1.1 -1.3, your group may develop a common response. For 1.4, each team member must provide an individual answer, using a different set of predictor and response variables.***

**Question 1.1** A step that can help in defining a project is writing down your response (dependent) and your feature/predictor (independent) variables. 

1. At this point, what do you think your response variable will be (i.e. what your model is trying to predict)?  
2. Based on this variable are you going to be doing regression or classification?
2. Does your response variable already exist in a dataset in the form that you need it?  
3. How many observations do you expect to have in your sample?  
4. What challenges have you encountered in acquiring the data?  

If you're choosing between a few options at this point, you can discuss more than one potential response variable.

*YOUR ANSWER HERE*

**Question 1.2** Questions to answer:  
1. What are the features that you might include in your model? 
2. Are there any features that you want to include, but you haven't figured out where to get or how to calculate the data? 
3. Where will the data for the features come from when the model is used for prediction? In other words, in this project, you will use observed $X$ and $y$ data to train, test, and cross-validate a model of the form $\hat {y_i} = \hat {f}(X_i)$. Ideally, you would then be able to use your model to predict $y$ in settings (e.g., locations, times) for which you *don't* have access to observations of the response variable. Are feature (i.e., $X$) data available for the settings in which you would want to predict $y$? 

*YOUR ANSWER HERE*

**Question 1.3** Based on your answer to the questions above, think about what constitutes an "ideal" observation of your data. Try to write out your data in the form

$$y_i = f(x_{1i}, x_{2i},...) + \epsilon_i$$

where $y_i$ is your response variable and $x_{1i}, x_{2i},...$ are your features. 

Specific questions to answer:

1. What is $y_i$, and what are its units? 
2. What are $x_{1i}, x_{2i},...$? 
3. What does $i$ index in your training data?  A point in time, a location in space? (Be as specific as you can!)
4. What work do you need to do to make $i$ index the same thing for your response varaible and your features?  As an example of the kind of "work" you'd need to do, it's often necessary to do some kind of aggregation (possibly using the `.resample` method in pandas), such as collapsing observations in time or space.  

As an example, if we were to answer the questions above for the Novotny et al. paper, we would get:<br>
1. $y$ is atmospheric $NO_2$ measurements in $\mu$g/m$^3$<br>
2. $x_1$ is % impervious surface<br>
$x_2$ is % tree canopy<br>
$x_3$ is population<br>
$x_4$ is major road length in km<br>
$x_5$ is minor road length in km<br>
$x_6$ is total road length in km<br>
$x_7$ is elevation in meteres<br>
$x_8$ is distance to coast in km<br>
$x_9$ is ground-based OMI $NO_2$ measurement in $\mu$g/m$^3$,<br>
and 
3. In the training data, $i$ indexes locations of the ground-based measurements.
4. Many of the features need to be aggregated into buffers around the ground-based measurements.  

*YOUR ANSWER HERE*

**Question 1.4** Choose two variables of interest from your dataset and visualize the relationship between them using whatever type of plot makes the most sense. Ideally, you would be able to explore the relationship between one of your predictors and your response variable, but if your response variable isn't finalized yet, you can take this as an opportunity to look at the relationship between any two interesting variables.

Comment on your plot - is the trend interesting, expected, or surprising at all?

*YOUR CODE OR VISUAL HERE*

*YOUR OBSERVATIONS HERE*

---

## Section 2: Overfitting <a name='of'></a>

In this class, we have been using mean squared error (MSE), which takes the following form, as a way of quantifying error from our models:

$$
\text{MSE} = \frac{1}{n}\sum\limits_{i=1}^{n}\Big(y_i-\hat{f}(x_i)\Big)^2
$$

where $y_i$ is our observed $y$ and $\hat{f}(x_i)$ is our estimator's best guess of the value $y$.

Our goal is to lower the MSE assocated with our model. However, in doing so, we risk overfitting if we minimize MSE on our data without considering how the model might perform on data that it hasn't seen before. As a basic example, we will try to choose a model with two parameters to fit linear data (with some noise added). By now, you should have a good idea of the variables that go into a linear model, and how to fine-tune them to decrease MSE.

**Dependencies:**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set_context("talk")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

np.random.seed(1)

We are going to create some pseudo-random data for the purposes of this homework. Read the block below before moving on and try to recite to yourself what each variable will represent. The answers for three of the variables are described below.

In [ ]:
# Run this block
n = 100
sigma_2 = 144
noise = np.random.normal(scale=np.sqrt(sigma_2), size=n)
X = np.random.uniform(-5, 5, n)
Y = 50 -2. * X  + 3 * X**2 + 1.5 * X**3 - 0.75 *X**4
Y_obs = Y + noise

* `n`: the number of datapoints in our space.
* `X`: the x-values of our data
* `Y`: is the non-biased estimator of our data (i.e. our response variable without any noise added). The model will roughly take its shape, but with a bit of added noise.

**Question 2.1:** Explain in 1-3 sentences what this line of code does. What do each of the arguments mean? How is the variable defined in this line used later on, and why is it important?

```python
np.random.normal(scale=np.sqrt(sigma_2), size=n)
```

*Hint:* This [link](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.normal.html) may help you.

*Your answer here*

We've now set up an experiment where we know the true relationship beteen `Y` and `X`, but we're going to try to predict that relationship using `Y_obs`, which are our observed values of `Y`. Let's start the prediction process with a visualization.


**Question 2.2:** Create a scatterplot of the values of `X` and `Y_obs`. You may need to make the dots smaller (or the plot bigger) to avoid too much crowding of datapoints. Be sure to give your plot a title and axis labels.

In [ ]:
# YOUR CODE HERE


**Question 2.3:** When fitting the coefficients to figure out the relationship between `Y_obs` and `X`, we'll start with a general model form. Below is the formula for a second-order model with three parameters.

$$ \Large
\hat{y_i} = \hat{\beta_0} + \hat{\beta_1} x_i + \hat{\beta_2} x_i^2
$$

In the block below, describe in one sentence what each variable represents. The first one is done as an example:

**ANSWER**:

* $\hat{y_i}$: This is the estimated value of the true $y$ based on our model's parameters.
* $\hat{\beta_0}$: YOUR ANSWER HERE
* $\hat{\beta_1}$: YOUR ANSWER HERE
* $\hat{\beta_2}$: YOUR ANSWER HERE
* $x_i$: YOUR ANSWER HERE

Now that we have a good grasp of what our dataset represents, we are going to try to find a model that best reproduces the relationship between $x$ and $y$. To do this, we need to split our data into train and test sets.

**Question 2.4:** Below, take our data `X` and `Y_obs` and create new data arrays for our new training and test sets. There should be 70% data inside of our training set, and 30% data in our test set. Use the `train_test_split()` function in the `sklearn.model_selection` library (also used in Lab 8) with a random state of 5.

In [ ]:
# YOUR CODE HERE
X_train, X_test, y_train, y_test = ...
assert [np.size(X_train), np.size(y_train)] == [70, 70]


**Question 2.5** Create a function, `scatter()`, that plots the training and testing data on the same scatter plot. As input, it should take in `X_train`, `X_test`, `y_train`, `y_test`, and a title. This will be useful later on, when we'll be outputting the scatter plot but overlaying different model fit lines on it - it will help to avoid copy-pasting code.

Use different markers or marker colors to denote training and testing data, and add a legend, axis labels, and a title (the title should change based on the input to the function).

After you've defined your function, call it in the second cell to produce a scatter plot of training and testing data.

In [ ]:
# solution
def scatter(X_train, X_test, y_train, y_test, title):
    # YOUR CODE HERE

In [ ]:
# RUN THIS CELL
scatter(X_train, X_test, y_train, y_test, "X vs Y_obs, divided into training and testing sets")

It's time to fit our first model! What if we weren't sure what the general model form looked like? Normally, you'd start with the most simple model and improve from there. So, that is exactly what we will do - we will create a ***feature matrix,*** $\Phi$. $\Phi$ is a matrix with $d$ columns, where each column contains the values of $X$ raised to a power from 1 to $d$. The general form of $\Phi$ looks like this:

$$ \large
\Phi_d(x) = \left[x, x^2, \ldots, x^d \right]
$$

For a first order model, $d$ would be equal to 1 and $\Phi$ would look like this:

$$ \large
\Phi_1(x) = \left[x \right]
$$


Similarly, for a second order model, $d$ would be equal to 2 and $\Phi$ would be:

$$ \large
\Phi_2(x) = \left[x, x^2 \right]
$$

Depending on how many features (degrees) we are adding to our model, we account for it using the following function:

In [ ]:
def poly_phi(d, X):
    #return lambda X: np.array([X ** i for i in range(1, k+1)]).T
    return np.array([X ** i for i in range(1, d+1)]).T

Before applying the function `poly_phi()` to our data, let's create a simple array `X_sample`. Run the cell below and make sure you understand how `poly_phi()` is working. Try changing the value for `d` to see how the output matrix changes for different degree values.

In [ ]:
X_sample = np.array([1,2,3])

poly_phi(1,X_sample)

**Question 2.6**: Define a function `lm_phi()` that performs a linear regression using a $\Phi_d$ matrix. 
* As input, it should take X_train, X_test, y_train, y_test, and d (the degree of matrix $\Phi_d$). 
* It should produce a scatter plot of the training and testing data (using the function `scatter()`) with a curve overlayed that shows the result of the linear regression. The plot's title should indicate the highest degree of the polynomial (e.g., "Order 1 Linear Model on Train and Test Data" for a first-order model). 
* It also should return the mean squared error of the test data and of the training data.

*Hint*: Here's the documentation for the `LinearRegression` object again: [docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

*Hint*: When it comes to plotting a the curve of a model in matplotlib, it sometimes make sense to create a range of `X` values, `X_plot`, then generate predictions for each of those. If you generate predictions for your `X_test` values and plot those, matplotlib will basically "connect the dots", but since the `X_test` values aren't sorted, you'll end up with a plot that has lines crossing each other. If, instead, you define a range of values `X_plot`, then generate predictions for each of those, you'll get a curve - matplotlib is still connecting the dots, but now that it's been given a sequential set of values, it will do so in a way that actually reflects the shape of the function. To get you started, we've defined a variable `X_plot` below that you can use to generate predictions.<br>

When you calculate the test MSE, however, you should be using `X_test` (and getting predictions based on those values) rather than `X_plot`.

In [ ]:
# SOLUTION
def lm_phi(d, X_train, X_test, y_train, y_test):
    phi_train = poly_phi(...) # get the phi matrix of the training data
    
    # fit a model using training data
    lm = ...
    lm.fit(...)
    
    X_plot = np.linspace(np.min(X_test), np.max(X_test), 200)
    
    y_pred = ...
    
    title = ... # plot title
    scatter(...) # create scatter plot
    plt.plot(...) # plot best fit curve
    
    MSE_test = ... # calculate MSE of test data
    MSE_train = ... # calculate MSE of training data
    
    return MSE_test, MSE_train

**Question 2.7**: Call `lm_phi()` with `d = 1` to fit a model using the $\Phi_1$ matrix. Print the testing and training MSEs.

Your figure will look something like this:
<img src='hw8_img.png' width="50%" height="50%"></img>

In [ ]:
# YOUR CODE HERE

Fitting the model with `d = 1` is a good start, but we're **underfitting** the model - we're not quite capturing the underlying trend of the data when we fit just the one feature. How do we fix this?

Let's try a different approach: instead of fitting just one feature, we'll fit many. Let's try creating a 32nd-order polynomial estimator. 

**Question 2.8**: Call `lm_phi()` with `d = 32` to fit a model using the $\Phi_{}$ matrix. Print the testing and training MSEs. After you've called `lm_phi()`, try using `plt.ylim()` to "zoom in and out" of your plot and get a better look at how the model varies with `X`, and output the plot with the y axis limits that best show how the model is responding to the training data.

In [ ]:
# YOUR CODE HERE

**Question 2.9** How did the values for training and test MSE change when you changed your feature matrix from having one column (`d = 1`) to 32 columns (`d = 32`)? What do the testing and training MSEs indicate about the relative bias and variance of the two models?

*Your answer here*

**Question 2.10** Try running `lm_phi()` with different values for `d`. Choose the model that provides (visually and based on MSE values) the best fit of training and testing data. 

*Note*: you don't have to systematically loop through all values for `d` and choose the optimal one - we'll do that in the next section. You can just try different values for `d` and settle for one - you will probably find that there's a small range of values for `d` that give roughly similar model fits.

In [ ]:
# YOUR CODE HERE

----

## Section 3: Cross Validation<a name='cv'></a>

You have now estimated the best fit for the data we created. Let's think about the steps we took to get to a successful answer: 

1. First, we started with a simple first-order polynomial to have a baseline training error.
1. We noticed from our graphing that it had high bias, so we continued to add more features...
1. ... until we reached a point in which both the training data and the test data would both be well-fit with the number of features we have chosen.

In Section 2, we were trying to visually figure out which model best fit both the training and test sets. However, we won't always be able to rely on just visualization for all cases. This is where k-fold cross validation (CV) comes in!


**k-fold cross validation** is a systematic way to make sure that we train the model at least once on every point in the dataset.

<img src='kfoldcv.png' width="50%" height="50%"></img>

The figure above, from ISLR pg. 181, shows an example of 5-fold cross validation. A k-fold cross validation algorithm randomly splits the data into k groups, ensuring that they're non-overlapping (i.e. each of the k groups has a validation set, and each validation set contains a distinct set of observations). In the figure above, each beige section represents about a fifth of the data set, and the model is fit 5 times with the remaining 4/5 of the data. We'll end up with 5 values for MSE, and we can calculate metrics like the average MSE or the variance of MSE to assess how the model responds to new data.

**Question 3.1:** Let's practice splitting training data into k-folds for validation purposes. Split the array `X` from Section 2 into 4 folds, shuffling before we add the batches, with a random state of 13. For each fold, print the indices of the Train and Validation sets onto the console.

Here's the [documentation for KFold()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).

In [ ]:
# Your code here

kf = KFold(...)

fold = 1
for train_i, val_i in kf.split(...):
    # print training indices
    # print validation indices
    fold+=1

You'll notice that each test set is distinct from the others, meaning that none of the test sets share common items even though they are shuffled pretty well. 

**Question 3.2:** Let's do a little review first before you start implementing the k-fold CV. Complete the function below to calculate the mean squared error between an array of predicted and true y-values.

In [ ]:
# ANSWER
def mse(y_pred, y_true):
    return ...

Recall back to Section 2 when you tried different numbers of features to train models. We can use k-fold CV to find out which model has the lowest mean squared error when predicting y-values. Let's define an array, `d_array`, of possible `d` values to evaluate:

In [ ]:
# run this cell
d_array = np.arange(1,32)

**Question 3.3:** Complete the function `mse_k_fold_lr` below. We are seeking to create a new k-fold CV pipeline that will give us the information we need to evaluate which features to include in our model based on the average cross-validated MSE of several models. 
* The inputs to the function include the number of folds (given by input `k`), the array of possible d values to evaluate (`d_array`, defined above), the `poly_phi` function defined in Section 2, the `mse` function defined in Question 3.2, the $X$ and $y$ arrays, and a random state given by the integer input `rand_state`. 
* Make sure to shuffle the indices when creating your folds.
* Your function should return a dictionary in which each key represents the degree of the highest polynomial in the model (i.e., the d-value associated with a model), and the value is the average value MSE associated with that model across all k folds. The dictionary should be saved in a variable called `average_mses`.<br>
* Your function should also return the d (degree) value that minimizes MSE across all models. 

This is a somewhat long function, but the code skeleton is written for you - focus on understanding what each step is trying to achieve (from the comments and from your understanding of k-fold cross-validation) and filling in code where you see ellipses (...).

In [ ]:
# ANSWER
def mse_k_fold_lr(k, d_array, poly_phi, mse, X, y, rand_state):
    '''This function will print the d (degree) value that gives the smallest MSE.
    Returns the dictionary of average MSEs (from 3-fold cross validation) across the specified d values
    '''
    
    # 1. K-Fold where k = k
    kf = KFold(...)
    
    # 2. Save the MSEs of each split.
    mses = np.full((k,len(d_array)),np.nan)
    # above, we're initializing an array where:
    # every row of mses corresponds to one of the folds
    # every column of mses corresponds to one of the possible d values in d_array
    # for example, mses[0,0] corresponds to the mean squared error for the first fold, using d = 1 (the first element of d_array)
    
    fold = 0 # initialize fold value
    for train_i, val_i in kf.split(X): # this loop iterates through the K folds of our data
        # 2.1 Separate X and Y_obs array into testing and validation sets
        X_fold_train = ...
        y_fold_train = ...
        X_fold_val = ...
        y_fold_val = ...
        
        # 2.2 Each d value will receive its own LinearRegression() model.
        for i in range(len(d_array)): # this loop iterates through d values to produce a new model for each d value
            # 2.2.1 Initialize the model
            model = ...
            
            # 2.2.2 Fit the training data using feature matrix with number of degrees equal to d[i] and the y_fold_train
            model.fit(...)
            
            # 2.2.3 Predict y-values for the test set of features transformed into a feature matrix
            y_pred = model.predict(...)
            
            # Save each mse between y_pred and y_fold_val at their respective fold and d value
            mses[...] = ...
        
        fold += 1 # augment the fold count
    
    # 3. Now, find the average of the MSEs for each d value. Your result should be a vector of the same length as d_array,
    # in which each element is the average MSE across all k folds associated with that d-value.
    average_mses = {}
    for i in range(len(d_array)):
        average_mses[d_array[i]] = ...
    
    # 4. Find the index of the minimum average MSE
    min_mse_index = min(average_mses, key=average_mses.get)
    
    print("Minimum MSE Parameters:", min_mse_index, '\n',
          "MSE of {} Parameters:".format(min_mse_index), average_mses[min_mse_index])
    return average_mses

In [ ]:
# call the function
average_mses = mse_k_fold_lr(4, d_array, poly_phi, mse, X_train, y_train, 13)
average_mses

**Question 3.4** The function call above for 4-fold cross validation on the training data with a random state of 13, should have returned a minimum MSE parameters of 4, with an MSE of about 144. In the cell below, try calling `mse_k_fold_lr()` but:

* changing the `rand_state` input but keeping `k` constant
* changing `k` to a larger value, and trying different values of `rand_state`

Then, answer the following questions:

* What are you changing in terms of the implementation of k-fold CV when you change `k`? How about when you change `rand_state`?
* Do the results (minimum MSE parameters and associated MSE) change when you vary `k` or `rand_state`?

In [ ]:
# scratch work

*Your answer here*

**Question 3.5:** Use the `matplotlib` or `seaborn` libraries to visualize the average *ROOT* mean squared error (RMSE) from cross-validating and to confirm the degree of the polynomial that minimizes MSE from Question 3.3. Use `k` = 4, `rand_state` = 13. Your plot should have a marker where the RMSE is minimized, and should have axis labels and a title.

*Hint*: you may want to plot the log of the RMSE to clearly display the full range of RMSE values

*Hint*: you can extract values from a dictionary using `.values()`, and can convert dictionary values to a list using `list()`.

In [ ]:
# YOUR CODE HERE

## Submission

Congrats, you're done with Homework 8!

Before you submit, click **Kernel** --> **Restart & Clear Output**. Then, click **Cell** --> **Run All**. Then, go to the toolbar and click **File** -> **Download as** -> **.html** and submit the file **as both an .html and .ipynb file through bCourses**.

----